In [1]:
%env WANDB_CACHE_DIR=./cache
%env HF_DATASETS_CACHE=./cache
%env TRANSFORMERS_CACHE=./cache
%env HF_HOME=./cache

# Credit: https://colab.research.google.com/github/m3hrdadfi/soxan/blob/main/notebooks/Emotion_recognition_in_Greek_speech_using_Wav2Vec2.ipynb#scrollTo=1SSs95o9WIvK
!pip install git+https://github.com/huggingface/datasets.git
!pip install git+https://github.com/huggingface/transformers.git
!pip install torchaudio
!pip install librosa
#!pip install -U accelerate
!pip install git+https://github.com/huggingface/accelerate
!pip install wandb

env: WANDB_CACHE_DIR=./cache
env: HF_DATASETS_CACHE=./cache
env: TRANSFORMERS_CACHE=./cache
env: HF_HOME=./cache
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/datasets.git to /scratch/6805556.1.ece/pip-req-build-ib0bds3l
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/datasets.git /scratch/6805556.1.ece/pip-req-build-ib0bds3l
  Resolved https://github.com/huggingface/datasets.git to commit f96e74d5c633cd5435dd526adb4a74631eb05c43
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/transformers.git to /scratch/6805556.1.ece/pip-req-build-c8jt8v2z
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/transf

Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
Defaulting to user installation because normal site-packages is not writeable
  Cloning https://github.com/huggingface/accelerate to /scratch/6805556.1.ece/pip-req-build-8ssl8_om
  Running command git clone --filter=blob:none --quiet https://github.com/huggingface/accelerate /scratch/6805556.1.ece/pip-req-build-8ssl8_om
  Resolved https://github.com/huggingface/accelerate to commit f7daaaa305362e1a959183e23bf85244798ceb35
  Installing build dependencies ... done
  Getting requirements to build wheel ... done
  Installing backend dependencies ... done
  Preparing metadata (pyproject.toml) ... done
Defaulting to user installation because normal site-packages is not writeable


In [1]:
%env WANDB_CACHE_DIR=./cache
%env HF_DATASETS_CACHE=./cache
%env TRANSFORMERS_CACHE=./cache
%env HF_HOME=./cache

import numpy as np
import pandas as pd
from pathlib import Path
from tqdm import tqdm
import accelerate
from packaging import version

from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report
import os
import sys
from datasets import load_dataset, load_metric

import transformers
from transformers import Trainer
from transformers import AutoConfig, Wav2Vec2Processor, EvalPrediction, TrainingArguments
from transformers.file_utils import ModelOutput
from transformers.models.wav2vec2.modeling_wav2vec2 import (Wav2Vec2PreTrainedModel, Wav2Vec2Model)
from transformers import (Trainer, is_apex_available)
if is_apex_available():
    from apex import amp

import librosa
from dataclasses import dataclass
from typing import Optional, Tuple, Dict, List, Union, Any

import torch
import torch.nn as nn
from torch.nn import CrossEntropyLoss
import torchaudio
if version.parse(torch.__version__) >= version.parse("1.6"):
    _is_native_amp_available = True
    from torch.cuda.amp import autocast
import torch.nn.functional as F

import IPython.display as ipd
from dataset import download_datasets, SpeechEmotionDataset, get_dataset_info

    
    
!nvidia-smi
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print(f"Device: {device}")


env: WANDB_CACHE_DIR=./cache
env: HF_DATASETS_CACHE=./cache
env: TRANSFORMERS_CACHE=./cache
env: HF_HOME=./cache


/usr4/ugrad/jhua2/.local/lib/python3.10/site-packages/transformers/utils/hub.py:124: FutureWarning: Using `TRANSFORMERS_CACHE` is deprecated and will be removed in v5 of Transformers. Use `HF_HOME` instead.
  warnings.warn(


Tue Apr 23 19:52:57 2024       
+---------------------------------------------------------------------------------------+
| NVIDIA-SMI 535.129.03             Driver Version: 535.129.03   CUDA Version: 12.2     |
|-----------------------------------------+----------------------+----------------------+
| GPU  Name                 Persistence-M | Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp   Perf          Pwr:Usage/Cap |         Memory-Usage | GPU-Util  Compute M. |
|                                         |                      |               MIG M. |
|=========================================+======================+======================|
|   0  NVIDIA A40                     On  | 00000000:17:00.0 Off |                    0 |
|  0%   38C    P0              73W / 300W |   1512MiB / 46068MiB |      0%   E. Process |
|                                         |                      |                  N/A |
+-----------------------------------------+----------------------+--

In [ ]:
'''
# Downloading datasets: execute once!
dataset_dir = './datasets'
os.makedirs(dataset_dir, exist_ok=True)
download_datasets(dataset_dir)
'''

In [2]:
dataset_dir = './datasets'
emo_map = {
    0 : "happy",
    1 : "anger",
    2 : "sadness",
    3 : "neutral"
}
emotions = list(emo_map.values())

# Extract data into dataframes
df = get_dataset_info(dataset_dir)

# Process data frame
df = df[['filename', 'emotion', 'database']]
df = df.rename(columns={'filename':'path'})
df['name'] = df['path'].str.split('/').str[-1]
df = df[df['emotion'].isin(emotions)]
df = df.reset_index(drop=True)

print("Example row from each database: \n", df.groupby('database').first())
print("Emotion labels: ", df['emotion'].unique())

# Test smaller size
df = df.sample(frac=1).reset_index(drop=True)
#df = df.iloc[:2000]
#df = df.iloc[:5000]
#df = df.iloc[:8000]
print("Number of samples: ", len(df))

Example row from each database: 
                                                        path  emotion  \
database                                                               
cremad       ./datasets/cremad/AudioWAV/1080_IEO_ANG_HI.wav    anger   
emodb     ./datasets/berlin-database-of-emotional-speech...  neutral   
ravdess   ./datasets/ravdess-emotional-speech-audio/Acto...  neutral   
savee        ./datasets/savee-database/AudioData/JE/a12.wav    anger   
shemo     ./datasets/shemo-persian-speech-emotion-detect...  sadness   
tess      ./datasets/toronto-emotional-speech-set-tess/T...    happy   

                              name  
database                            
cremad         1080_IEO_ANG_HI.wav  
emodb                  16a02Nb.wav  
ravdess   03-01-01-01-02-01-03.wav  
savee                      a12.wav  
shemo                   F06S01.wav  
tess            OAF_nice_happy.wav  
Emotion labels:  ['neutral' 'anger' 'sadness' 'happy']
Number of samples:  10548


In [3]:
train_df, test_df = train_test_split(df, test_size=0.2, random_state=101, stratify=df["emotion"])
train_df, test_df = train_df.reset_index(drop=True), test_df.reset_index(drop=True)
train_df.to_csv("train.csv", sep="\t", encoding="utf-8", index=False)
test_df.to_csv("test.csv", sep="\t", encoding="utf-8", index=False)

print("train_df's shape: ", train_df.shape)
print("test_df's shape: ", test_df.shape)

data_files = {
    "train": "train.csv",
    "validation": "test.csv"
}

dataset = load_dataset("csv", data_files=data_files, delimiter="\t")
train_dataset = dataset["train"]
eval_dataset = dataset["validation"]
print(train_dataset, eval_dataset)

num_labels = len(emo_map)
label_list = list(emo_map.values())
print(f"{num_labels} classes: {label_list}")

train_df's shape:  (8438, 4)
test_df's shape:  (2110, 4)


Generating train split: 0 examples [00:00, ? examples/s]

Generating validation split: 0 examples [00:00, ? examples/s]

Dataset({
    features: ['path', 'emotion', 'database', 'name'],
    num_rows: 8438
}) Dataset({
    features: ['path', 'emotion', 'database', 'name'],
    num_rows: 2110
})
4 classes: ['happy', 'anger', 'sadness', 'neutral']


In [4]:
model_name = "lighteternal/wav2vec2-large-xlsr-53-greek" # todo: use an english or multilingual one
label2id = {label:id for id,label in emo_map.items()}

config = AutoConfig.from_pretrained(
    model_name,
    num_labels = num_labels,
    label2id = label2id,
    id2label = emo_map, 
    finetuning_task = "wav2vec2_clf" # classification task
)
setattr(config, "pooling_mode", "mean")

processor = Wav2Vec2Processor.from_pretrained(model_name)
target_sampling_rate = processor.feature_extractor.sampling_rate
print(f"Target sampling rate: {target_sampling_rate}")

Target sampling rate: 16000


/usr4/ugrad/jhua2/.local/lib/python3.10/site-packages/transformers/configuration_utils.py:363: UserWarning: Passing `gradient_checkpointing` to a config initialization is deprecated and will be removed in v5 Transformers. Using `model.gradient_checkpointing_enable()` instead, or if you are using the `Trainer` API, pass `gradient_checkpointing=True` in your `TrainingArguments`.
  warnings.warn(


In [5]:
# Preprocess data

def load_audio(path):
    speech_array, sampling_rate = librosa.load(path)
    speech_array = speech_array.squeeze()
    
    # Ensure audio has expected sampling rate
    audio = librosa.resample(np.asarray(speech_array), orig_sr = sampling_rate, target_sr = target_sampling_rate)
    return audio

def preprocess(data):
    speech_list = [load_audio(path) for path in data["path"]]
    target_list = [label2id[emo] for emo in data["emotion"]]
    
    preprocessed_audio = processor(speech_list, sampling_rate=target_sampling_rate)
    preprocessed_audio["labels"] = list(target_list)
    
    return preprocessed_audio

train_dataset = train_dataset.map(
    preprocess,
    batch_size=100,
    batched=True,
    num_proc=4
)

eval_dataset = eval_dataset.map(
    preprocess,
    batch_size=100,
    batched=True,
    num_proc=4
)
    

Map (num_proc=4):   0%|          | 0/8438 [00:00<?, ? examples/s]

Map (num_proc=4):   0%|          | 0/2110 [00:00<?, ? examples/s]

In [6]:
# Classification component of model

class Classification_FC(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.dense = nn.Linear(config.hidden_size, config.hidden_size)
        self.dropout = nn.Dropout(config.final_dropout)
        self.out_proj = nn.Linear(config.hidden_size, config.num_labels)
    
    def forward(self, features, **kwargs):
        x = features
        
        x = self.dropout(x)
        
        x = self.dense(x)
        
        x = torch.tanh(x)
        
        x = self.dropout(x)
        
        x = self.out_proj(x)
        
        return x

class Classification_CNN(nn.Module):
    def __init__(self, config):
        super().__init__()
        
        # 1D CNN
        self.conv1 = nn.Conv1d(in_channels=1, out_channels=16, kernel_size=3)
        self.conv2 = nn.Conv1d(in_channels=16, out_channels=32, kernel_size=3)
        self.pool = nn.MaxPool1d(kernel_size=2)
        
        # Fully connected layers
        self.dense1 = nn.Linear(8128, 4096)
        self.dense2 = nn.Linear(4096, 1024)
        self.out_proj = nn.Linear(1024, config.num_labels)
        
        self.dropout = nn.Dropout(config.final_dropout)
    
    def forward(self, features, **kwargs):
        x = features.unsqueeze(1) # [4, 1, 1024]

        # 1D CNN layers
        x = self.pool(torch.relu(self.conv1(x))) # torch.Size([4, 16, 511])
        
        x = self.pool(torch.relu(self.conv2(x))) # torch.Size([4, 32, 254])
        
        x = torch.flatten(x, 1) # torch.Size([4, 8128])

        # FC layers
        
        x = self.dropout(x)
        
        x = self.dropout(torch.relu(self.dense1(x)))
        
        x = self.dropout(torch.relu(self.dense2(x)))
        
        x = self.out_proj(x)
        
        return x
    
class Classification_LSTM(nn.Module):
    def __init__(self, config):
        super().__init__()
        self.num_layers = 4
        self.hidden_dim = 512
        
        self.lstm = nn.LSTM(1024, self.hidden_dim, num_layers=self.num_layers, dropout=0.2, batch_first=True)
        self.dropout = nn.Dropout(config.final_dropout)
        self.fc = nn.Linear(512, 128)
        self.out_proj = nn.Linear(128, config.num_labels)
        
    def init_hidden(self):
        hidden = torch.zeros(self.num_layers, self.hidden_dim).to(device)
        cell = torch.zeros(self.num_layers, self.hidden_dim).to(device)
        return hidden, cell
    
    def detach_hidden(self, hidden):
        hidden, cell = hidden
        hidden, cell = hidden.detach(), cell.detach()
        return hidden, cell
    
    def forward(self, features, hidden, **kwargs):
        x = features
        
        x, hidden = self.lstm(x, hidden)
        
        x = self.dropout(x)
        
        x = self.dropout(torch.relu(self.fc(x)))
        
        x = self.out_proj(x)
        
        return x, hidden

In [7]:
# Model
@dataclass
class SpeechClassifierOutput(ModelOutput):
    loss: Optional[torch.FloatTensor] = None
    logits: torch.FloatTensor = None
    hidden_states: Optional[Tuple[torch.FloatTensor]] = None
    attentions: Optional[Tuple[torch.FloatTensor]] = None

    
class SER_Model(Wav2Vec2PreTrainedModel):
    def __init__(self, config, classification_type="fc"):
        super().__init__(config)
        self.num_labels = config.num_labels
        self.pooling_mode = config.pooling_mode
        self.config = config
        
        self.wav2vec2 = Wav2Vec2Model(config)
        
        # Choose classifier
        self.classifier = Classification_FC(config)
        self.classification_type = classification_type
        if classification_type == "cnn":
            self.classifier = Classification_CNN(config)
        elif classification_type == "lstm":
            self.classifier = Classification_LSTM(config)
            self.hidden = self.classifier.init_hidden()
        
        
        self.init_weights()
        self.loss_fct = CrossEntropyLoss()
    
    def freeze_feature_extractor(self):
        self.wav2vec2.feature_extractor._freeze_parameters()
        
    def merged_strategy(
        self,
        hidden_states,
    ):
        # Mode: mean
        outputs = torch.mean(hidden_states, dim=1)
        return outputs

    def forward(
        self,
        input_values,
        attention_mask=None,
        output_attentions=None,
        output_hidden_states=None,
        return_dict=None,
        labels=None
    ):
        return_dict = return_dict if return_dict is not None else self.config.use_return_dict
        outputs = self.wav2vec2(
            input_values,
            attention_mask = attention_mask,
            output_attentions = output_attentions,
            output_hidden_states = output_hidden_states,
            return_dict = return_dict
        )
        hidden_states = outputs[0]
        hidden_states = self.merged_strategy(hidden_states)
        
        logits = self.classifier(hidden_states)
        
        if self.classification_type == 'lstm':
            logits, self.hidden = self.classifier(hidden_states, self.hidden) # lstm
            self.hidden = self.classifier.detach_hidden(self.hidden)
        else:
            logits = self.classifier(hidden_states)
        
        
        loss = self.loss_fct(logits, labels)
        
        if not return_dict: #???
            output = (logits,) + outputs[2:]
            return ((loss,) + output)
        
        return SpeechClassifierOutput(
            loss = loss,
            logits = logits,
            hidden_states = outputs.hidden_states,
            attentions = outputs.attentions
        )
        
        
        
        
    

In [8]:
# REVIEW!!!
@dataclass
class DataCollatorCTCWithPadding:
    processor: Wav2Vec2Processor
    padding: Union[bool, str] = True
    max_length: Optional[int] = None
    max_length_labels: Optional[int] = None
    pad_to_multiple_of: Optional[int] = None
    pad_to_multiple_of_labels: Optional[int] = None

    def __call__(self, features: List[Dict[str, Union[List[int], torch.Tensor]]]) -> Dict[str, torch.Tensor]:
        input_features = [{"input_values": feature["input_values"]} for feature in features]
        label_features = [feature["labels"] for feature in features]

        d_type = torch.long if isinstance(label_features[0], int) else torch.float

        batch = self.processor.pad(
            input_features,
            padding=self.padding,
            max_length=self.max_length,
            pad_to_multiple_of=self.pad_to_multiple_of,
            return_tensors="pt",
        )

        batch["labels"] = torch.tensor(label_features, dtype=d_type)

        return batch
    
data_collator = DataCollatorCTCWithPadding(processor=processor, padding=True)


In [9]:
def compute_metrics(p: EvalPrediction):
    preds = p.predictions[0] if isinstance(p.predictions, tuple) else p.predictions
    preds = np.argmax(preds, axis=1)
    return {"accuracy": (preds == p.label_ids).astype(np.float32).mean().item()}

In [10]:
training_args = TrainingArguments(
    output_dir = "./test_outputs/",
    per_device_train_batch_size=2,
    per_device_eval_batch_size=2,
    gradient_accumulation_steps=2,
    evaluation_strategy="steps",
    num_train_epochs=10.0,
    fp16=True,
    save_steps=1000,
    eval_steps=1000,
    logging_steps=1000,
    learning_rate=1e-4,
    save_total_limit=2,
)

/usr4/ugrad/jhua2/.local/lib/python3.10/site-packages/transformers/training_args.py:1449: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(


In [11]:
%env WANDB_WATCH=all
%env WANDB_LOG_MODEL=1
%env WANDB_PROJECT="Speech-Emotion-Recognition"
!wandb login 2f06d46d1776b669cf43407c6d32e3e34fafdb29 --relogin

env: WANDB_WATCH=all
env: WANDB_LOG_MODEL=1
env: WANDB_PROJECT="Speech-Emotion-Recognition"
wandb: Appending key for api.wandb.ai to your netrc file: /usr4/ugrad/jhua2/.netrc


In [12]:
model = SER_Model.from_pretrained(
    model_name,
    config = config, 
    classification_type = "fc"
)
model.freeze_feature_extractor()

trainer = Trainer(
    model = model,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor
)

trainer.train()

Some weights of SER_Model were not initialized from the model checkpoint at lighteternal/wav2vec2-large-xlsr-53-greek and are newly initialized: ['classifier.dense.bias', 'classifier.dense.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: jaylinlily123. Use `wandb login --relogin` to force relogin


Step,Training Loss,Validation Loss,Accuracy
1000,0.812000,0.899499,0.735545
2000,0.587200,0.544841,0.851659
3000,0.481000,0.526368,0.868246
4000,0.446100,0.479834,0.868720
5000,0.383600,0.527172,0.870142
6000,0.311900,0.543197,0.872038
7000,0.273000,0.537007,0.882464
8000,0.248600,0.596102,0.886256
9000,0.213100,0.634240,0.882938
10000,0.200300,0.529269,0.896682


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


TrainOutput(global_step=21090, training_loss=0.23403594779877823, metrics={'train_runtime': 10081.189, 'train_samples_per_second': 8.37, 'train_steps_per_second': 2.092, 'total_flos': 9.796257349490024e+18, 'train_loss': 0.23403594779877823, 'epoch': 9.997629770087698})

In [13]:
torch.save(model.state_dict(), 'wav2vec2_ser_fc.pt')

In [16]:
# CNN as classification head
training_args = TrainingArguments(
    output_dir = "./test_outputs/",
    per_device_train_batch_size=1,
    per_device_eval_batch_size=1,
    gradient_accumulation_steps=1,
    evaluation_strategy="steps",
    num_train_epochs=3.0,
    fp16=True,
    save_steps=8438,
    eval_steps=8438,
    logging_steps=8438,
    learning_rate=1e-4,
    save_total_limit=2,
)

model_cnn = SER_Model.from_pretrained(
    model_name,
    config = config, 
    classification_type = "cnn"
)
model_cnn.freeze_feature_extractor()

trainer_cnn = Trainer(
    model = model_cnn,
    data_collator=data_collator,
    args=training_args,
    compute_metrics=compute_metrics,
    train_dataset=train_dataset,
    eval_dataset=eval_dataset,
    tokenizer=processor.feature_extractor
)

trainer_cnn.train()

/usr4/ugrad/jhua2/.local/lib/python3.10/site-packages/transformers/training_args.py:1449: FutureWarning: `evaluation_strategy` is deprecated and will be removed in version 4.46 of 🤗 Transformers. Use `eval_strategy` instead
  warnings.warn(
Some weights of SER_Model were not initialized from the model checkpoint at lighteternal/wav2vec2-large-xlsr-53-greek and are newly initialized: ['classifier.conv1.bias', 'classifier.conv1.weight', 'classifier.conv2.bias', 'classifier.conv2.weight', 'classifier.dense1.bias', 'classifier.dense1.weight', 'classifier.dense2.bias', 'classifier.dense2.weight', 'classifier.out_proj.bias', 'classifier.out_proj.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.
Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


Step,Training Loss,Validation Loss,Accuracy
8438,1.501700,1.218934,0.735545
16876,1.222900,1.173993,0.773934
25314,0.964800,0.993728,0.806161


Detected kernel version 4.18.0, which is below the recommended minimum of 5.5.0; this can cause the process to hang. It is recommended to upgrade the kernel to the minimum version or higher.


TrainOutput(global_step=25314, training_loss=1.2297895078439796, metrics={'train_runtime': 5033.3787, 'train_samples_per_second': 5.029, 'train_steps_per_second': 5.029, 'total_flos': 2.6041270238980444e+18, 'train_loss': 1.2297895078439796, 'epoch': 3.0})

In [17]:
torch.save(model_cnn.state_dict(), 'wav2vec2_ser_cnn_3epochs.pt')

In [81]:
test_dataset = load_dataset("csv", data_files={"test": "test.csv"}, delimiter="\t")["test"]
model = model.to(device) # model_cnn?

def predict(speech, label):
    features = processor(speech, sampling_rate=target_sampling_rate, return_tensors="pt", padding=True)
    
    input_values = features.input_values.to(device)
    attention_mask = features.attention_mask.to(device)
    labels = torch.tensor([label]).to(device)
    print(labels.size(), input_values.size())
    
    with torch.no_grad():
        logits = model(input_values, attention_mask = attention_mask, labels=labels).logits
        
    scores = F.softmax(logits, dim=1).detach().cpu().numpy()[0]
    outputs = [{"Emotion": config.id2label[i], "Score": f"{round(score * 100, 3):.1f}%"} for i, score in enumerate(scores)]
    return outputs


def prediction(df_row):
    path, emotion = df_row["path"], df_row["emotion"]
    df = pd.DataFrame([{"Emotion": emotion, "Sentence": "    "}])
    setup = {
        'border': 2,
        'show_dimensions': True,
        'justify': 'center',
        'classes': 'xxx',
        'escape': False,
    }
    speech, label = load_audio(path), label2id[emotion]
    outputs = predict(speech, label)
    r = pd.DataFrame(outputs)
    
    ipd.display(ipd.Audio(data=np.asarray(speech), rate=target_sampling_rate))
    print(r)
    print("Label: ", emotion)
    #ipd.display(ipd.HTML(STYLES + r.to_html(**setup) + "<br />"))

In [83]:
test = pd.read_csv("test.csv", sep="\t")
prediction(test.iloc[1])

torch.Size([1]) torch.Size([1, 66201])


     Emotion  Score
0    neutral   1.2%
1       calm   0.0%
2      happy  15.0%
3        sad   0.0%
4      angry  34.4%
5    fearful   0.1%
6    disgust   4.2%
7  surprised  45.0%
Label:  surprised


In [ ]:
# Test accuracy on each dataset